<a href="https://colab.research.google.com/github/jodog0412/DACON/blob/main/SHOPPING_DATA_ANALYSIS_BEGINNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab
'[Baseline]_1. 데이터 분석 입문자를 위한 데이터 분석 & 예측.ipynb'
'[Baseline]_2. 랜덤 포레스트를 사용한 데이터 분석 & 예측.ipynb'
 dataset
 SHOPPING_DATA_ANALYSIS_BEGINNER.ipynb


# 데이터 로드

In [ ]:
import pandas as pd
import numpy as np

# 데이터 로드
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')
sample_submission = pd.read_csv('./dataset/sample_submission.csv')


# 전처리

## train 전처리

In [ ]:
# 결측치 처리
train=train.fillna(0)

# Date 전처리
def date_encoder(date):
    day, month, year = map(int, date.split('/'))
    return day, month, year
train['Month'] = train['Date'].apply(lambda x: date_encoder(x)[1])
train['Year'] = train['Date'].apply(lambda x: date_encoder(x)[2])
# isHoliday column에서의 true, false는 1과 0으로 자동처리됨.

from sklearn.preprocessing import StandardScaler

# 정규화
scaler = StandardScaler()
scaler.fit(train[['Promotion1',
                  'Promotion2',
                  'Promotion3',
                  'Promotion4',
                  'Promotion5']])
scaled = scaler.transform(train[['Promotion1',
                                 'Promotion2',
                                 'Promotion3',
                                 'Promotion4',
                                 'Promotion5']])
train[['Scaled_Promotion1','Scaled_Promotion2',
       'Scaled_Promotion3','Scaled_Promotion4',
       'Scaled_Promotion5']] = scaled

# 분석할 의미가 없는 칼럼을 제거합니다.
x_train = train.drop(columns=['id','Date','Weekly_Sales'])
x_train = x_train.drop(columns=['Promotion1',
                                'Promotion2',
                                'Promotion3',
                                'Promotion4',
                                'Promotion5'])
y_train = train[['Weekly_Sales']]
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Store              6255 non-null   int64  
 1   Temperature        6255 non-null   float64
 2   Fuel_Price         6255 non-null   float64
 3   Unemployment       6255 non-null   float64
 4   IsHoliday          6255 non-null   bool   
 5   Month              6255 non-null   int64  
 6   Year               6255 non-null   int64  
 7   Scaled_Promotion1  6255 non-null   float64
 8   Scaled_Promotion2  6255 non-null   float64
 9   Scaled_Promotion3  6255 non-null   float64
 10  Scaled_Promotion4  6255 non-null   float64
 11  Scaled_Promotion5  6255 non-null   float64
dtypes: bool(1), float64(8), int64(3)
memory usage: 543.8 KB


## test 전처리

In [ ]:
# 결측치 처리
test = test.fillna(0)
# Date 전처리
test['Month'] = test['Date'].apply(lambda x: date_encoder(x)[1])
test['Year'] = test['Date'].apply(lambda x: date_encoder(x)[2])
# 정규화
scaled = scaler.transform(test[['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']])
test[['Scaled_Promotion1','Scaled_Promotion2',
       'Scaled_Promotion3','Scaled_Promotion4',
       'Scaled_Promotion5']] = scaled
test = test.drop(columns=['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5'])
test = test.drop(columns=['id','Date'])
test

,Store,Temperature,Fuel_Price,Unemployment,IsHoliday,Month,Year,Scaled_Promotion1,Scaled_Promotion2,Scaled_Promotion3,Scaled_Promotion4,Scaled_Promotion5
0,1,68.55,3.617,6.573,False,10,2012,0.956583,-0.170211,-0.079007,0.686920,0.509844
1,1,62.99,3.601,6.573,False,10,2012,-0.053718,-0.170211,-0.080897,-0.106760,1.068815
2,1,67.97,3.594,6.573,False,10,2012,-0.245241,-0.170211,-0.081492,-0.244199,0.190716
3,1,69.16,3.506,6.573,False,10,2012,0.030535,-0.163792,-0.081292,0.012961,-0.054187
4,2,70.27,3.617,6.170,False,10,2012,0.612584,-0.170211,-0.080536,0.531594,0.565067
...,...,...,...,...,...,...,...,...,...,...,...,...
175,44,46.97,3.755,5.217,False,10,2012,-0.398965,-0.169683,-0.082230,-0.265323,-0.260073
176,45,64.89,3.985,8.667,False,10,2012,0.445482,-0.170211,-0.078895,0.327864,0.197316
177,45,54.47,4.000,8.667,False,10,2012,-0.075621,-0.170211,-0.080938,-0.107560,0.598392
178,45,56.47,3.969,8.667,False,10,2012,-0.067571,-0.170211,-0.081819,-0.150097,0.002305


# 딥러닝

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

# model = nn.Sequential(
#     nn.Linear(12, 24), # input_layer = 12, hidden_layer1 = 24
#     nn.ReLU(),
#     nn.Linear(24, 12), # hidden_layer1 = 24, hidden_layer2 = 12
#     nn.ReLU(),
#     nn.Linear(12, 6), # hidden_layer2 = 12, hidden_layer3 = 6
#     nn.ReLU(),
#     nn.Linear(6, 3), # hidden_layer4 = 6, hidden_layer3 = 3
#     nn.ReLU(),
#     nn.Linear(3, 1) # hidden_layer2 = 3, hidden_layer3 = 1
# )
x_train_val=np.vstack(x_train.values).astype(np.float64)
y_train_val=np.vstack(y_train.values).astype(np.float64)
X = torch.tensor(x_train_val)
Y = torch.tensor(y_train_val)
model=nn.Linear(12,1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.245e-6)
losses = []
epochs=25000
torch.set_grad_enabled(True)
for epoch in range(epochs+1):
    y_pred = model(X.float()) 
    loss = F.mse_loss(y_pred, Y.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

test_val=test.values.astype(np.float64)
T=torch.tensor(test_val)
model.eval()
prediction=model(T.float())
print(prediction[:10])

tensor([[1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875],
        [1047619.1875]], grad_fn=<SliceBackward0>)


In [ ]:
sample_submission['Weekly_Sales'] = prediction.detach().numpy()
sample_submission.head()
sample_submission.to_csv('submission.csv',index = False)